In [12]:
#load pickle file
import pickle
import numpy as np
import seaborn as sns

In [13]:
with open('../Data_Preprocessing/sphere.pkl', 'rb') as f:
        sphere = pickle.load(f)

with open('../Data_Preprocessing/no.pkl', 'rb') as f:
        no = pickle.load(f)

with open('../Data_Preprocessing/vort.pkl', 'rb') as f:
        vort = pickle.load(f)



In [14]:
 #make  y values for all the data class

y_sphere = [0 for i in range(len(sphere))]
y_no = [1 for i in range(len(no))]
y_vort = [2 for i in range(len(vort))]

#append all the y values
y = y_sphere + y_no + y_vort

#append all x values
x  =  np.concatenate((sphere, no, vort), axis=0)



### Preparation of Input Data


In [15]:
#for every data point in numpy x of 3 dim covert it to log
# from tqdm import tqdm
# for i in tqdm(range(len(x)),desc = 'Converting to log',total=len(x)):
#     for j in range(len(x[i])):
#         for k in range(len(x[i][j])):
#             x[i][j][k] = np.log(x[i][j][k])


x = np.log(x)

In [16]:
x = np.transpose(x, (0, 2, 1))

In [17]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)


In [18]:
#split the data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
x_train.shape

(24000, 360, 24)

### LSTM Architecture

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Dropout, Flatten

# Update parameters based on the revised structure
n_steps = 360  # Sequence length
n_features = 24  # Number of features at each timestep
out_len = 3  # Number of output classes for classification


model = Sequential()

# First Conv1D layer
model.add(Conv1D(filters=50, kernel_size=8, strides=2, activation='relu', input_shape=(n_steps, n_features)))

# Second Conv1D layer
model.add(Conv1D(filters=25, kernel_size=4, strides=2, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.2))

# LSTM layer to process the now flattened sequence
model.add(LSTM(150))

# Additional dropout to prevent overfitting
# model.add(Dropout(0.3))

# A Dense layer for feature learning
model.add(Dense(50, activation='relu'))

# Final Dense layer with softmax for classification
model.add(Dense(out_len, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Displaying the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 177, 50)           9650      
                                                                 
 conv1d_5 (Conv1D)           (None, 87, 25)            5025      
                                                                 
 lstm_2 (LSTM)               (None, 150)               105600    
                                                                 
 dense_4 (Dense)             (None, 50)                7550      
                                                                 
 dense_5 (Dense)             (None, 3)                 153       
                                                                 
Total params: 127,978
Trainable params: 127,978
Non-trainable params: 0
_________________________________________________________________


### Model Stopped Early due to poor results

In [26]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
750/750 [==============================] - 13s 16ms/step - loss: nan - accuracy: 0.3332 - val_loss: nan - val_accuracy: 0.3337
Epoch 2/10
750/750 [==============================] - 11s 14ms/step - loss: nan - accuracy: 0.3332 - val_loss: nan - val_accuracy: 0.3337
Epoch 3/10
575/750 [======================>.......] - ETA: 2s - loss: nan - accuracy: 0.3333

### Conclusion

#### The model is not performing well so AUC score wasnt calculated